
### Reglas generales del modelo
-   Dos nonine no pueden ocupar la misma sección.
-   Dos deddian no pueden ocupar la misma sección.
-   Cada nonine solo puede comer hierba si no ha llegado a su máxima capacidad de comida.
-   Un deddian no se comerá un nonine si ha sobrepasado su capacidad máxima de comida.
-   Nonines y deddians solo puede reproducirse cuando han llegado a cierta edad y ha logrado una cierta capacidad de cimida.
-   Un nonine no puede reproducirse si hay un deddian a su alrededor.
-   Nonines y deddians solo pueden mover hacia arriba, abajo, izquierda o derecha de su posición actual.
-   Nonines y deddians mueren si son muy viejos o si no han comido lo suficiente.
-   La hierba tiene una tasa de crecimiento de 1 unidad/ciclo.

### Reglas para los nonine
-   Capacidad máxima de comida: 45 unidades.
-   Tasa de metabolismo: 3 unidades/ciclo.
-   Probilidad de reproducción en un ambiente adecuada: 50%
-   Edad mínima de reproducción: 10 ciclos.
-   Requisito mínimo de alimento para reproducirse: 40 unidades.
-   Edad máxima: 25 etapas.
-   Comida inicial: 10
-   Un nonine le da un valor de comida de 10 a un deddian cuando se lo come.

En cada ciclo,
-   Un nonine se mueve aleatoriamente a uno de los cuatro campos vecinos, siempre que esté vacío (no existe otro nonine o deddian). Este movimiento le consume una cierta cantidad de la comida que ha ingerido. Si no hay celdas vecinas libres, no se produce movimiento.
-   Si no ha llegado a su capacidad máxima de comida, consumirá toda lo que hay en ese campo (hasta el límite definido).
-   Al superar su tiempo de reproducción predefidino, el nonine dará a a luz un nuevo nonine siempre y cuando tenga la energía necesaria y esté en un ambiente adecuado. El nuevo nonine se deberá generar en una celda vecina vacía. Si no se cumple alguno de estos requisitos, no se generará ningún nuevo nonine.

### Reglas para los deddian

-   Capacidad máxima de comida: 200 unidades.
-   Tasa de metabolismo: 2 unidades/ciclo.
-   Probilidad de reproducción en un ambiente adecuada: 50%
-   Edad mínima de reproducción: 10 ciclos.
-   Requisito mínimo de alimento para reproducirse: 120 unidades.
-   Edad máxima: 50 etapas.
-   Comida inicial: 150

En cada ciclo,

-   Un deddian se mueve aleatoriamente a uno de los cuatro campos vecinos, siempre que no haya otro deddian. Este movimiento le consume una cierta cantidad de la comida que ha ingerido. Si no hay celdas vecinas libres de deddians, no se produce movimiento.
-   Si se encuentra con un nonine y no ha sobrepasado su capacidad máxima, se lo comerá. Obteniendo una cantidad de energía definida.
-   Al superar su tiempo de reproducción predefidino, el deddian dará a a luz un nuevo deddian siempre y cuando tenga la energía necesaria y esté en un ambiente adecuado. El nuevo deddian se deberá generar en una celda vecina vacía. Si no se cumple alguno de estos requisitos, no se generará ningún nuevo deddian.

## Condiciones de simulación
-   El planeta tenga un ancho y alto de 35 celdas.
-   Inicialmente, habrá 47 nonines y 15 deddians.
-   Todos los campos empiezan con 20 unidades de hierba.
-   La simulación se realizará durante 200 iteraciones.
-   Deberá visualizar la evolución del modelo.

In [17]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Con ''RandomActivation'', hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

MAX_ITERATIONS = 200

In [18]:
def get_grid(model):
    """ Esta función nos permite obtener el estado de los diferentes agentes.
        *param* model : Modelo del que obtendrá la información. 
        *return* una matriz con la información del estado de cada uno de los agentes."""
    grid = np.zeros( (model.grid.width, model.grid.height) )
    for (content, x, y) in model.grid.coord_iter():
        if (content == None):
            grid[x][y] = 0
        else:
            grid[x][y] = content.type
    return grid

In [19]:
class NonineAgent(Agent):
    def __init__(self, unique_id, model, max_food = 45, metabolism = 3, reproduction_rate = 0.5, 
                 min_age_reproduction = 10, min_food_reproduction = 40, max_age = 25, initial_food = 10):
        super().__init__(unique_id, model)
        self.max_food = max_food
        self.metabolism = metabolism
        self.reproduction_rate = reproduction_rate
        self.min_age_reproduction = min_age_reproduction
        self.min_food_reproduction = min_food_reproduction
        self.max_age = max_age
        self.initial_food = initial_food
        self.current_food = initial_food
        self.current_age = 1
        
    def step():
        available_cells = []
        move_to_cell = 0
        neighborhood = self.get_neighborhood(self.pos, moore=False, include_center=False, radius=1)
        total_nonines = 0
        total = 0
        neighbors = self.get_neighbors(self.pos, moore=False, include_center=False, radius=1)
        deddians_close = False
        
        if self.current_age >= self.max_age or self.current_food <= 0:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
            return
        
        if self.model.grid[self.pos[0]][self.pos[1]] == 1:
            if self.current_food <= (self.max_food - self.model.grass_hunger):
                self.current_food += self.model.grass_hunger
                self.model.grid[self.pos[0]][self.pos[1]] = 0
        
        for neighbor in neighbors:
            if type(neighbor) is type(self):
                total_nonines += 1
            else:
                deddians_close = True
            total += 1
        
        fraction = total_nonines / total
        
        for cell in neighborhood:
            if self.model.grid.is_cell_empty(cell):
                available_cells.append(cell)
        
        if self.current_age >= self.min_age_reproduction and self.current_food >= self.min_food_reproduction:
            #if fraction >= self.reproduction_rate and len(available_cells) > 0:
            if deddians_close == True and len(available_cells) > 0:
                new_nonine = NonineAgent(self.unique + 500, self.model)
                new_nonine_pos = available_cells[random.randint(0, len(available_cells) - 1)]
                self.model.grid.place_agent(new_nonine, new_nonine_pos)
                self.model.schedule.add(new_nonine)
                available_cells.remove(new_nonine_pos)
                
        if len(available_cells) > 0 and self.current_food >= self.metabolism:
            self.current_food -= self.metabolism
            move_to_cell = available_cells[random.randint(0, len(available_cells) - 1)]
            self.model.grid.move_agent(self, move_to_cell)

# Preguntar si el la tasa de reproducción se refiere a que hay un 50|50 de que se reproduzca cuando se cumplen todas las condiciones
# para reproducirse

# Preguntar si puede haber más de un agente en una misma celda (considerando que son diferentes agentes)

In [20]:
class DeddianAgent(Agent):
    def __init__(self, unique_id, model, max_food = 200, metabolism = 2, reproduction_rate = 0.5, 
                 min_age_reproduction = 10, min_food_reproduction = 120, max_age = 50, initial_food = 150):
        super().__init__(unique_id, model)
        self.max_food = max_food
        self.metabolism = metabolism
        self.reproduction_rate = reproduction_rate
        self.min_age_reproduction = min_age_reproduction
        self.min_food_reproduction = min_food_reproduction
        self.max_age = max_age
        self.initial_food = initial_food
        
    def step():
        print("Hola")

In [21]:
class CenituneModel(Model):
    def __init__(self, width = 35, height = 35, num_grass = 20, num_nonines = 47, num_deddians = 15):
        self.num_grass = num_grass
        self.num_nonines = num_nonines
        self.num_deddians = num_deddians
        self.grid = SingleGrid(width, height, false)
        self.schedule = RandomActivation(self)
        self.grass_hunger = 5